# Hyper-parameter Optimization
Wrap __[hyperopt](https://hyperopt.github.io/hyperopt/)__ into **HyperParameterOpt** to do hyper-parameter optimization. Use random search and logged the parameters have been searched.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from hyperopt import hp, fmin, tpe, Trials, space_eval, STATUS_OK
# from schema.columns_added_filled import LABEL_COL, NUMERICAL_COLS, CATEGORICAL_COLS, LOG_COLS
from schema.columns_added import LABEL_COL, NUMERICAL_COLS, CATEGORICAL_COLS, LOG_COLS
from data_process.data_transform_processor import DataTransformProcessor
from models.nn_models.dnn import DNN
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet
from models.tree_models.lgbm import LGBM
from models.hyperparameter_opt import HyperParameterOpt
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


# Prepare data

In [2]:
df_all = pd.read_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/df_merged_20171008.csv')

In [3]:
data_processor_dct = {
    'dummy': DataTransformProcessor(
    use_dummy=True, use_scale=True,
    numerical_cols=NUMERICAL_COLS, 
    categorical_cols=CATEGORICAL_COLS,
    log_cols=LOG_COLS, 
    label_col=LABEL_COL,
),
    'tree': DataTransformProcessor(
    numerical_cols=NUMERICAL_COLS, 
    categorical_cols=CATEGORICAL_COLS,
    log_cols=LOG_COLS, 
    label_col=LABEL_COL,
),
    'dummy_pca': DataTransformProcessor(
    use_dummy=True, use_scale=True, use_pca=True,
    numerical_cols=NUMERICAL_COLS, 
    categorical_cols=CATEGORICAL_COLS,
    log_cols=LOG_COLS, 
    label_col=LABEL_COL,
),
    'tree_pca': DataTransformProcessor(
    use_scale=True, use_pca=True,
    numerical_cols=NUMERICAL_COLS, 
    categorical_cols=CATEGORICAL_COLS,
    log_cols=LOG_COLS, 
    label_col=LABEL_COL,
),
}

In [4]:
# use all data_processors to pre process to get categorical idx
for processor in data_processor_dct.values():
    X_all = processor.pre_process(df_all)
y_all = df_all['logerror'].values

# Define search space

## Search space for DNN

In [5]:
space_dnn = {
    'learning_rate': hp.loguniform('learning_rate', -4*np.log(10), -1*np.log(10)),
    'reg': hp.loguniform('reg', -4*np.log(10), -1*np.log(10)),
    'decay': hp.loguniform('decay', -1*np.log(10), -3*np.log(10)),
#     'dim_hidden_lst': hp.choice('dim_hidden_lst', [(20,), (30,)]),
}

## Search space for LightGBM

In [6]:
space_gbdt = {
    'learning_rate': hp.loguniform('learning_rate', -2*np.log(10), -1*np.log(10)),  # 1e-2 ~ 1e-1
    'n_estimators': 50 * (10 + hp.randint('n_estimators', 11)),  # 500 ~ 1000
    'num_leaves': 20 * (10 + hp.randint('num_leaves', 11)),  # 200 ~ 400
    'lambda_l2': hp.loguniform('lambda_l2', 1*np.log(10), 4*np.log(10)),  # 1e0 ~ 1e4
    'feature_fraction': hp.uniform('feature_fraction', 0.3, 0.8),  #  0.3 ~ 0.8
    'bagging_freq': 10 * (2 + hp.randint('bagging_freq', 7)),  # 20 ~ 80
    'bagging_fraction': hp.uniform('bagging_fraction', 0.7, 1.0),
    'max_bin': 40 * (1 + hp.randint('max_bin', 10)),  # 40 ~ 400
}

space_dart = space_gbdt.copy()
space_dart.update({
    'drop_rate': hp.uniform('drop_rate', 0.0, 1.0),  #  0.0 ~ 1.0
    'skip_drop':hp.uniform('skip_drop', 0.0, 1.0),  #  0.0 ~ 1.0,
})
                              
fixed_params_gbdt_l1 = {
    'objective': 'regression_l1',
    'boosting_type': 'gbdt',
    'categorical_feature': data_processor_dct['tree'].categorical_col_idx,
}

fixed_params_gbdt_l2 = {
    'objective': 'regression_l2',
    'boosting_type': 'gbdt',
    'categorical_feature': data_processor_dct['tree'].categorical_col_idx,
}

fixed_params_dart_l1 = {
    'objective': 'regression_l1',
    'boosting_type': 'dart',
    'categorical_feature': data_processor_dct['tree'].categorical_col_idx,
}

## Search space for Linear Regression ElasticNet

In [7]:
space_eln = {
    'alpha': hp.loguniform('alpha', -2*np.log(10), 2*np.log(10)),
    'l1_ratio': hp.uniform('l1_ratio', 0.0, 1.0),
}

# Optimize for hyper-parameters

In [8]:
hyper_opt = HyperParameterOpt(
    model_class=LGBM, 
    data_processor=data_processor_dct['tree'],                           
    search_space=space_gbdt,
    fixed_params=fixed_params_gbdt_l1,
    max_evals=60,
    max_workers=4,
)

In [9]:
%%time
hyper_opt.optimize(X_all, y_all, 12)

Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV fi

In [12]:
hyper_opt.trial_results

,bagging_fraction,bagging_freq,feature_fraction,lambda_l2,learning_rate,max_bin,n_estimators,num_leaves,loss,status
0,0.969946,60,0.505398,2951.160086,0.021524,400,1000,260,0.067758,ok
1,0.916933,30,0.694704,597.604114,0.022082,40,650,220,0.067762,ok
2,0.919865,30,0.522237,1444.233321,0.017810,240,950,380,0.067767,ok
3,0.930635,30,0.694073,1096.446102,0.016553,40,650,220,0.067767,ok
4,0.896889,80,0.638455,3777.741119,0.021738,160,600,240,0.067769,ok
5,0.978923,80,0.496287,2608.567056,0.026410,400,700,260,0.067769,ok
6,0.871400,60,0.543539,322.883244,0.019744,320,1000,200,0.067770,ok
7,0.954269,40,0.642728,3159.268998,0.028855,40,500,320,0.067771,ok
8,0.942883,30,0.749066,556.806989,0.027453,40,650,220,0.067774,ok
9,0.903785,60,0.562554,268.384105,0.016475,120,600,220,0.067774,ok


# Save opt log history

In [11]:
hyper_opt.trial_results.to_csv(
    '/Users/shuyangdu/Desktop/ZillowChallenge/hyper-parameter-opt/gbdt_l1_added_features_201701010.csv',
)